In [1]:
import re
import pandas as pd
pd.set_option('display.max_rows', None)
with open('output-grid-causal-2.txt') as fhand:
    file = fhand.read()
op = re.split(r'\n\n\n',file)
op[0]

'parameter(adv_step, adv_epoch, adv_epoch_full, lr, ro): (2048.0, 10.0, 10.0, 1e-06, 1e-06)\ngender/race combined consistency 0\nspouse consistency 0\nacc_temp: 0.39115533443891654, bal_acc_temp: 0.4883667542293745, race_gap_rms_temp: 0.007396053779178127, race_max_gap_temp: 0.008405974510597547, gender_gap_rms_temp: 0.18381555276619255, gender_max_gap_temp: 0.2108603170665478'

In [2]:
def string_format(x):
    s = re.findall(r'(\d+)\.(\d+)|(\d+)e-(\d+)|(\d)\.(\d+)e-(\d+)|(\d+)',x)
    string_s = []
    for i in s:
        if i[0] != '':
            string_s.append(i[0]+'.'+i[1])
        if i[2] != '':
            string_s.append(i[2] + 'e-'+i[3])
        if i[4] != '':
            string_s.append(i[4]+'.'+i[5]+'e-'+i[6])
        if i[7] !='':
            string_s.append(i[7])
    return(string_s)
ops = [string_format(x) for x in op]
ops = ops[:(len(ops)-1)]

In [3]:
output_dict = dict()
for op in ops:
    for i in range(len(op)):
        op[i] = float(op[i])
    output_dict[tuple(op[:5])] = op[5:]

In [4]:
df_output = pd.DataFrame.from_dict(output_dict, orient='index', columns=['gender/race combined consistency','spouse consistency', 'acc-temp' , 'bal-acc-temp', 'race-gap-rms', 'race-max-gap', 'gender-gap-rms', 'gender-max-gap'])
df_output = df_output.rename_axis('(adv_step, adv_epoch, adv_epoch_full, lr, ro)')
df_output = df_output.sort_values(by=['race-gap-rms', 'race-max-gap', 'gender-gap-rms', 'gender-max-gap'])
df_output = df_output.sort_values(by=['bal-acc-temp'],ascending=False)

Outputs for adult data with sensr algorithm. Sensetive directions (directions for both gender and race) are from causal regression: X_gender_race_train ~ y_gender_race_train

In [5]:
df_output[['acc-temp' , 'bal-acc-temp', 'race-gap-rms', 'race-max-gap', 'gender-gap-rms', 'gender-max-gap']]

,acc-temp,bal-acc-temp,race-gap-rms,race-max-gap,gender-gap-rms,gender-max-gap
"(adv_step, adv_epoch, adv_epoch_full, lr, ro)",,,,,,
"(4.0, 10.0, 80.0, 0.01, 0.0001)",0.800111,0.823348,0.060730,0.082622,0.145268,0.195247
"(4.0, 10.0, 10.0, 0.01, 1e-06)",0.811830,0.822601,0.054582,0.073675,0.124589,0.172748
"(4.0, 10.0, 10.0, 0.01, 0.01)",0.805860,0.822052,0.055112,0.077926,0.154946,0.199518
"(4.0, 10.0, 80.0, 0.01, 0.01)",0.811277,0.818994,0.058521,0.075355,0.135760,0.175071
"(4.0, 10.0, 10.0, 0.01, 0.0001)",0.809287,0.818451,0.046482,0.060431,0.115795,0.160284
"(4.0, 10.0, 10.0, 0.0001, 0.01)",0.788612,0.787662,0.037187,0.040811,0.037138,0.047540
"(32.0, 10.0, 80.0, 0.0001, 1e-06)",0.779104,0.785394,0.019070,0.026701,0.029665,0.037414
"(4.0, 10.0, 80.0, 0.0001, 0.01)",0.771808,0.782734,0.026589,0.033316,0.025334,0.033658
"(4.0, 10.0, 10.0, 0.0001, 0.0001)",0.774572,0.774991,0.036266,0.049477,0.037419,0.051858
